# null=0 + fbprophet Trend 적용 + year,month,day 변수 추가

In [1]:
from prophet import Prophet

import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import random
from copy import deepcopy

In [2]:
train_df=pd.read_csv('./data/train.csv')
test_df=pd.read_csv('./data/test.csv')

In [3]:
train_df_backup = deepcopy(train_df)
test_df_backup = deepcopy(test_df)

In [4]:
df=train_df[['date','rental']].copy()

In [5]:
df.columns=['ds','y']

In [6]:
m = Prophet()
m.daily_seasonality=True
m.weekly_seasonality=True
m.yearly_seasonality=True
m.fit(df)

Initial log joint probability = -73.9282


In [7]:
future=m.make_future_dataframe(365)
forecast = m.predict(future)

    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1699.14   5.17586e-06       69.4858      0.7698      0.7698      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       1699.17   0.000410967       93.1272   3.446e-06       0.001      181  LS failed, Hessian reset 
     150       1699.19   3.83621e-05       77.7678   4.552e-07       0.001      266  LS failed, Hessian reset 
     177       1699.19    1.5188e-07       69.0376      0.2292           1      304   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [8]:
random_state_list=[random.randint(0, 100) for _ in range(10)]
random_state_list

[74, 72, 88, 26, 30, 21, 72, 37, 95, 55]

In [9]:
train_df = deepcopy(train_df_backup)
train_df.fillna(0, inplace=True)
train_df['rental']=train_df['rental']/forecast[:-365]['trend']

In [10]:
train_df2 = train_df.copy()
train_df2['date']=pd.to_datetime(train_df2['date'])
train_df2['year']=train_df2['date'].dt.year
train_df2['month']=train_df2['date'].dt.month
train_df2['day']=train_df2['date'].dt.day
train_df2 = train_df2[['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'year', 'month', 'day', 'rental']]

In [11]:
train_df.columns

Index(['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'rental'],
      dtype='object')

In [12]:
train_df2.columns

Index(['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'year', 'month', 'day', 'rental'],
      dtype='object')

In [13]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X.shape, y.shape

((1095, 11), (1095,))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.7894562704519024
Oob_Score : 0.8139583472211821
2번째
Score : 0.7870634032105902
Oob_Score : 0.8146219990319563
3번째
Score : 0.7908139752823177
Oob_Score : 0.8142221927751937
4번째
Score : 0.7936947954173483
Oob_Score : 0.8134374960854501
5번째
Score : 0.7966474454672765
Oob_Score : 0.8147867179788962
6번째
Score : 0.7863357161285509
Oob_Score : 0.8171190039026354
7번째
Score : 0.7870634032105902
Oob_Score : 0.8146219990319563
8번째
Score : 0.7872070535901928
Oob_Score : 0.8114850384188683
9번째
Score : 0.7915627816133148
Oob_Score : 0.8129508819634605
10번째
Score : 0.7884326706451575
Oob_Score : 0.8154358835338973


In [15]:
X,y = train_df2.iloc[:,1:-1].to_numpy(), train_df2.iloc[:,-1].to_numpy()
X.shape, y.shape

((1095, 14), (1095,))

In [16]:
print(*sorted(zip(train_df.iloc[:,1:-1].columns ,rf.feature_importances_), key=lambda x : x[1], reverse=True), sep='\n')

('temp_highest', 0.5630478885189943)
('precipitation', 0.10519306431282438)
('humidity', 0.08764142518903276)
('PM2.5', 0.04556753592694212)
('sunshine_sum', 0.041824841934523016)
('sunshine_rate', 0.04121132019734954)
('PM10', 0.030312790976585974)
('temp_mean', 0.024137617306773607)
('wind_mean', 0.022679812598662495)
('wind_max', 0.02153847029388419)
('temp_lowest', 0.016845232744427577)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.9042178133794752
Oob_Score : 0.9180106245441485
2번째
Score : 0.9068163054344015
Oob_Score : 0.9137115784438639
3번째
Score : 0.9079119210704482
Oob_Score : 0.9158890874964585
4번째
Score : 0.9064376369743773
Oob_Score : 0.9162837060307586
5번째
Score : 0.9058940483088813
Oob_Score : 0.9157270042192932
6번째
Score : 0.9068724292845468
Oob_Score : 0.9130976442404221
7번째
Score : 0.9068163054344015
Oob_Score : 0.9137115784438639
8번째
Score : 0.9080071695668182
Oob_Score : 0.9149211190195207
9번째
Score : 0.9032593262745735
Oob_Score : 0.9153042841709284
10번째
Score : 0.9051269966897862
Oob_Score : 0.9115581327472074


In [18]:
print(*sorted(zip(train_df2.iloc[:,1:-1].columns ,rf.feature_importances_), key=lambda x : x[1], reverse=True), sep='\n')

('temp_highest', 0.5403805451019141)
('year', 0.12296218139327067)
('precipitation', 0.10247848416327782)
('humidity', 0.06777060759010575)
('temp_mean', 0.04050351623265926)
('sunshine_rate', 0.03455839587543247)
('sunshine_sum', 0.021833683747155087)
('temp_lowest', 0.017982006449004086)
('month', 0.01486642051235831)
('PM10', 0.008616209496355633)
('PM2.5', 0.008213190583780311)
('day', 0.007697831656042833)
('wind_max', 0.006676440872589329)
('wind_mean', 0.005460486326054206)
